In [ ]:
!pip install ktrain
import ktrain
from ktrain import text

In [ ]:
import pandas as pd

df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

In [ ]:
from sklearn.model_selection import train_test_split

y = df.pop('Emotion')
X = df

X_train,X_test,y_train,y_test = train_test_split(X.to_numpy().ravel() ,y.to_numpy().ravel(), test_size=0.05, stratify=y)

X_train.shape
y_test.shape

In [ ]:
a = pd.DataFrame(y_test)
a.value_counts()

In [ ]:
Emotions = ['anger', 'disgust', 'fear', 'guilt', 'joy', 'sadness', 'shame'] #classes for Isear
#Emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'] #classes for Meld-DailyDialog

#Choose the model you want
MODEL_NAME = 'distilbert-base-uncased'
#MODEL_NAME = 'bert-base-uncased'
#MODEL_NAME = 'albert-base-v2'

t = text.Transformer(MODEL_NAME, maxlen=200, class_names=Emotions)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=24) 

In [ ]:
lr_list = [5e-5, 4e-5, 3e-5, 2e-5]
epoch_list = [2,3,4]

#class_weight = {0: 6, 1: 22, 2: 30, 3: 1, 4: 2, 5: 7, 6: 5} #class weights for Meld-dd dataset

for lr in lr_list:
  for epoch in epoch_list:
    print("Learning_rate: " + str(lr))
    print("Epoch: " + str(epoch))
    learner.fit_onecycle(lr, epoch) #Add class_weight = class_weight as parameter if training Meld-dd dataset
    print("Results")
    learner.validate(class_names=t.get_classes())
    print("------------------------------")

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
#the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [ ]:
#print(X_test[229])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.save('mgsa-ed')
print('MODEL SAVED')

In [ ]:
sentence = 'I am the luckiest man alive'

In [ ]:
predictor.predict(sentence)

In [ ]:
predictor.predict_proba(sentence)

In [ ]:
predictor.get_classes()